# Proyecto Opti

### Importar librerias y datos

In [ ]:
import numpy as np
from gurobipy import Model, GRB, quicksum
from load_data import (L, N, G, E, P, R, T, L_r, L_e, N_e,
                       costo_l, plazo_l, 
                       gen_l, emp_l, ubi_l, tec_l, cap_e, REQ,
                       max_r, costo_n, plazo_n, emp_n, ubi_n)


{'Eólica', 'Hidro', 'Solar'}


### Crear modelo

In [15]:
def modelo():

    model = Model("GeneracionElectrica")
    model.setParam('TimeLimit', 1800)


    #x[l, t] vale 1 si se realiza el proyecto l en el semestre t
    x = model.addVars(L, T, vtype=GRB.BINARY, name="x")

    #y[l, t] vale 1 si el proyecto l esta en construccion en el semestre t
    y = model.addVars(L, T, vtype=GRB.BINARY, name="y")

    #w[n, t] vale 1 si se realiza el pryecto n en el semestre t
    w = model.addVars(N, T, vtype=GRB.BINARY, name="w")

    #z[n, t] vale 1 si el proyecto n esta en construccion en el semestre t
    z = model.addVars(N, T, vtype=GRB.BINARY, name="z")

    print("Variables añadidas")


    model.update()

    #Restricciones

    #1. La capacidad de generaci´on el´ectrica total es suficiente para satisfacer la demanda energ´etica proyectada para el 2050
    model.addConstr(quicksum((x[l, t] * gen_l[l]) for l in L for t in T) >= REQ)
    print("R1 añadida")

    #2. Todos lo proyectos deben terminarse a mas tardar en diciembre de 2049
    model.addConstrs(x[l, t] * (t + plazo_l[l]) <= 50 for l in L for t in T)
    model.addConstrs(w[n, t] * (t + plazo_n[n]) <= 50 for n in N for t in T)
    print("R2 añadida")

    #3. Una empresa solo puede desarrollar tantos proyectos paralelamente como le permite su capacidad.
    #model.addConstrs(quicksum(y[l, t] * emp_l[l, e] for l in L) + quicksum(z[n, t] * emp_n[n, e] for n in N) <= cap_e[e] for e in E for t in T)

    #L_por_empresa = {e: [] for e in E}
    #N_por_empresa = {e: [] for e in E}

    #for e in E:
    #    for l in L:
    #        if emp_l.get((l, e), 0) == 1:
    #            L_por_empresa[e].append(l)
    #            break 
    #    for n in N:
    #        if emp_n.get((n, e), 0) == 1:
    #            N_por_empresa[e].append(n)
    #            break 

    model.addConstrs((quicksum(y[l, t] for l in L_e[e]) + quicksum(z[n, t] for n in N_e[e])) <= cap_e[e] for e in E for t in T)

    print("R3 añadida")

    #4. No pueden realizarse 2 proyectos en la misma ubicaci´on. N´otese que esto evita que un proyecto se construya 2 veces
    model.addConstrs(quicksum(x[l, t] * ubi_l[l, p] for l in L for t in T) <= 1 for p in P)
    model.addConstrs(quicksum(w[n, t] * ubi_n[n, p] for n in N for t in T) <= 1 for p in P)
    print("R4 añadida")


    #5. Para hacer un proyecto de transmisión debe haber uno de generación asociado
    #? Revisar
    #model.addConstrs(quicksum(x[l, t] * ubi_l[l, p] for l in L for t in T) <= quicksum(w[n, t] * ubi_n[n, p] for n in N for t in T) + quicksum(ubi_n[n_prima, p] for n_prima in N) for p in P)
    model.addConstrs(quicksum(x[l, t] * ubi_l[l, p] for l in L for t in T) <= quicksum(w[n, t] * ubi_n[n, p] for n in N for t in T) for p in P)
    print("R5 añadida")


    #6. Comportamiento de yℓ,t y de z
    model.addConstrs(x[l, t] <= y[l, t + t_prima] for l in L for t in T for t_prima in range(plazo_l[l]) if t + t_prima <= T[-1]) #revisar lo del if t + t_prima
    model.addConstrs(quicksum(y[l, t] for t in T) == quicksum(x[l, t] * plazo_l[l] for t in T) for l in L)

    model.addConstrs(w[n, t] <= z[n, t + t_prima] for n in N for t in T for t_prima in range(plazo_n[n]) if t + t_prima < T[-1]) #revisar lo del if t + t_prima
    model.addConstrs(quicksum(z[n, t] for t in T) == quicksum(w[n, t] * plazo_n[n] for t in T) for n in N)  #falot el para todo t en modelación?
    print("R6 añadida")


    #7. No construir más proyectos de cierta tecnología que los que te permite la regi´on
    #model.addConstrs(quicksum(x[l, t] * tec_l[l, g] * ubi_l[l, p] for l in L for t in T for p in P) <= max[r, g] for r in R for g in G)

    #proyectos_por_zona = {}
    #for l in L:
    #    region = proyectos_g[l].Region
    #    for g in G:
    #        if tec_l.get((l, g), 0) == 1:
    #            if (region, l) not in proyectos_por_zona:
    #                proyectos_por_zona[(region, g)] = []
    #            proyectos_por_zona[(region, g)].append(l)

    model.addConstrs((quicksum(x[l, t] * tec_l[l, g] for l in L_r[r] for t in T)) <= max_r[r, g] for r in R for g in G)

    print("R7 añadida")


    #¬    #8. No superar la capacidad de transmisión de cada línea
    #¬    model.addConstrs(quicksum(w[n, t] * trans_n[n] * ubi_n[n, p] for t in range(T) for p in range(P)) >= quicksum(gen_l[l] * ubi_l[l, p] for l in range(L) for p in range(P)) for n in N)

    #Función objetivo
    #¬ model.setObjective(quicksum(costo_g[l, t] * x[l, t] for l in L for t in T) + quicksum(w[n, t] * costo_g[n, t] for n in N for t in T), GRB.MINIMIZE)
    model.setObjective(quicksum(costo_l[l] * x[l, t] for l in L for t in T) + quicksum(w[n, t] * costo_n[n] for n in N for t in T), GRB.MINIMIZE)
    
    model.update()
    print("Empezando a optimizar...")
    model.optimize()

    if model.status == GRB.OPTIMAL: return model

### Ejecutar modelo

In [16]:
model = modelo()

Set parameter TimeLimit to value 1800
Variables añadidas
R1 añadida
R2 añadida
R3 añadida
R4 añadida
R5 añadida
R6 añadida
R7 añadida
Empezando a optimizar...
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 752146 rows, 240500 columns and 2002822 nonzeros
Model fingerprint: 0x7bd0e2fe
Variable types: 0 continuous, 240500 integer (240500 binary)
Coefficient statistics:
  Matrix range     [2e-01, 3e+03]
  Objective range  [1e-01, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+100]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 640101 rows and 37679 columns
Presolve time: 4.85s
Presolved: 112045 rows, 202821 columns, 970726 nonzeros
Variable types: 0 continuous, 202821 integer (202739 

### Imprimir resultados

In [18]:
print(model.status == GRB.OPTIMAL)

True
